Pre-process the input data

In [ ]:
import re
import pandas as pd
from collections import Counter

df = pd.read_csv("data/pokemon_entries.csv", encoding="utf-16-le", sep="\t")

# Filter for gen 1
df = df.loc[df["gen"] == "I"]

# Lowercase everything
df["english_name"] = df["english_name"].str.lower()
df["description"] = df["description"].str.lower()
df["classification"] = df["classification"].str.lower()

# Append type and classification to description
df["description"] = df["primary_type"] + " " + df["classification"].replace(" pokemon", "") + " " + df["description"]

columns_to_keep = [
    "english_name",
    "description",
]
df = df[columns_to_keep]

# Replace all pronouns by the pokemon nouns
def transform(row):
    pokemon_name = row["english_name"]
    return (
        row["description"]
        .replace("’s", ' is')
        .replace("'s", ' is')
        .replace("it", f' {pokemon_name}')
        .replace("its", pokemon_name)
        .replace("this pokémon", pokemon_name)
    )
df["description"] = df.apply(lambda row: transform(row), axis=1)

df

In [55]:
# Count the number of words
def unique_words_counts(text):
    # Remove punctuation marks
    text = re.sub(r'[^\w\s]', '', text)
    # Use regular expression to find all words
    words = re.findall(r'\b\w+\b', text.lower())
    # Count the occurrences of each word
    word_counts = Counter(words)
    return word_counts

# Extract all description strings
descriptions = df["description"].values
single_string = " ".join(descriptions)
unique_word_count = unique_words_counts(single_string)
print(f"Number of unique words: {len(unique_word_count)}")

Number of unique words: 1469


In [66]:
save_path = "data/pokemon_cleansed.csv"
df.to_csv(save_path, index=False)